<a href="https://colab.research.google.com/github/ncsu-landscape-dynamics/LAMP_assay_automation/blob/main/MaskRCNN_2nd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://towardsdatascience.com/mask-rcnn-implementation-on-a-custom-dataset-fd9a878123d4

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#!git clone --quiet https://github.com/John-Polo/Mask_RCNN.git

In [2]:
!git clone --quiet https://github.com/matterport/Mask_RCNN.git

In [2]:
#!rm -r /content/Mask_RCNN/

rm: cannot remove '/content/Mask_RCNN/': No such file or directory


In [2]:
%cd /content/Mask_RCNN
!pip install -q PyDrive
!pip install -r requirements.txt
!python setup.py install

/content/Mask_RCNN
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:694: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:694: UserWarning: Usage of dash-separated 'license-file' will not be supported in future versions. Please use the underscore name 'license_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:694: UserWarning: Usage of dash-separated 'requirements-file' will not be supported in future versions. Please use the underscore name 'requirements_file' instead
  % (opt, underscore_opt))
running install
running bdist_egg
running egg_info
writing mask_rcnn.egg-info/PKG-INFO
writing dependency_links to mask_rcnn.egg-info/dependency_links.txt
writing top-level names to mask_rcnn.egg-info/top_level.txt
reading manifest template 'MANIFES

In [3]:
#!pip uninstall tensorflow
!pip install h5py==2.10.0
!pip install tensorflow==1.15.0
!pip install tensorflow-gpu==1.15.0

In [4]:
#!pip uninstall gast
!pip install gast==0.3.2

Processing /root/.cache/pip/wheels/59/38/c6/234dc39b4f6951a0768fbc02d5b7207137a5b1d9094f0d54bf/gast-0.3.2-cp37-none-any.whl
ERROR: tensorflow 1.15.0 has requirement gast==0.2.2, but you'll have gast 0.3.2 which is incompatible.
ERROR: tensorflow-gpu 1.15.0 has requirement gast==0.2.2, but you'll have gast 0.3.2 which is incompatible.
ERROR: kapre 0.3.5 has requirement tensorflow>=2.0.0, but you'll have tensorflow 1.15.0 which is incompatible.
  Found existing installation: gast 0.2.2
    Uninstalling gast-0.2.2:
      Successfully uninstalled gast-0.2.2


In [5]:
!pip uninstall keras-nightly
!pip install keras==2.1.6

In [ ]:
import tensorflow
print (tensorflow.__version__)

1.15.3


In [6]:
#!pip install mrcnn

In [7]:
#import mrcnn

In [47]:
import os
import sys
import itertools
import math
import datetime
import logging
import json
import re
import skimage.draw
import cv2
import torch
import torch.utils.data
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Directory to save logs and model checkpoints
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

%matplotlib inline

In [7]:
class ChambersConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "c_shapes"

    # Train on 1 GPU and 1 images per GPU. Batch size is 1 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 2  # background + #? shapes

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)  # anchor side in pixels

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 16

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 25

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 5
    
config = ChambersConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  512
IMAGE_META_SIZE                15
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [512 512   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE             

In [60]:
class chambersDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):
        """Load a subset of the Horse-Man dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
        self.add_class("object", 1, "bubble")
        self.add_class("object", 2, "chamber")
        # self.add_class("object", 3, "xyz") #likewise

        # Train or validation dataset?
        assert subset in ["train", "validation"]
        dataset_dir = os.path.join(dataset_dir, subset)

        # Load annotations
        # VGG Image Annotator saves each image in the form:
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'polygon'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }
        # We mostly care about the x and y coordinates of each region
        annotations1 = json.load(open(os.path.join(dataset_dir, "annotations.json")))
        # print(annotations1)
        annotations = list(annotations1.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]
        
        # Add images
        for a in annotations:
            # print(a)
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. There are stores in the
            # shape_attributes (see json format above)
            polygons = [r['shape_attributes'] for r in a['regions']] 
            objects = [s['region_attributes']['region_type'] for s in a['regions']]
            print("objects:",objects)
            name_dict = {"bubble": 1,"chamber": 2} #,"xyz": 3}
            # key = tuple(name_dict)
            num_ids = [name_dict[a] for a in objects]
     
            # num_ids = [int(n['Event']) for n in objects]
            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            print("numids",num_ids)
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object",  ## for a single class just add the name here
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a Horse/Man dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
        	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

        	mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        # Map class names to class IDs.
        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [ ]:
#annotations1 = json.load(open(os.path.join('/content/drive/MyDrive/APHIS Farm Bill (2020Milestones)/Protocols/For John/images/New set for John/collection/four_chambers/train', "annotations.json")))
#        # print(annotations1)
#annotations = list(annotations1.values())  # don't need the dict keys
#
#        # The VIA tool saves images in the JSON even if they don't have any
#        # annotations. Skip unannotated images.
#annotations = [a for a in annotations if a['regions']]
#        
#        # Add images
#for a in annotations:
#            # print(a)
#            # Get the x, y coordinaets of points of the polygons that make up
#            # the outline of each object instance. There are stores in the
#            # shape_attributes (see json format above)
#            polygons = [r['shape_attributes'] for r in a['regions']] 
#            objects = [s['region_attributes']['detail'] for s in a['regions']]
#            print("objects:",objects)
#            name_dict = {"bubble": 1,"chamber": 2} #,"xyz": 3}
#            # key = tuple(name_dict)
#            num_ids = [name_dict[a] for a in objects]

In [61]:
def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = chambersDataset()
    dataset_train.load_custom('/content/drive/MyDrive/APHIS Farm Bill (2020Milestones)/Protocols/For John/images/New set for John/collection/four_chambers', subset = 'train')
    dataset_train.prepare()

    # Validation dataset
    dataset_val = chambersDataset()
    dataset_val.load_custom('/content/drive/MyDrive/APHIS Farm Bill (2020Milestones)/Protocols/For John/images/New set for John/collection/four_chambers',subset = 'validation')
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=10,
                layers='heads')

In [62]:
config = ChambersConfig()
model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=DEFAULT_LOGS_DIR)

weights_path = COCO_WEIGHTS_PATH
        # Download weights file
if not os.path.exists(weights_path):
  utils.download_trained_weights(weights_path)

model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])

In [63]:
train(model)

objects: ['bubble', 'bubble', 'bubble', 'bubble', 'bubble', 'chamber', 'chamber', 'chamber', 'chamber', 'bubble', 'bubble', 'bubble', 'bubble', 'bubble', 'bubble', 'bubble']
numids [1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1]
objects: ['bubble', 'bubble', 'bubble', 'chamber', 'chamber', 'chamber', 'chamber', 'bubble', 'bubble', 'bubble', 'bubble', 'bubble', 'bubble']
numids [1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1]
objects: ['bubble', 'bubble', 'bubble', 'bubble', 'bubble', 'bubble', 'chamber', 'chamber', 'chamber', 'chamber', 'bubble', 'bubble', 'bubble', 'bubble', 'bubble', 'bubble']
numids [1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1]
objects: ['bubble', 'bubble', 'bubble', 'chamber', 'chamber', 'chamber', 'chamber', 'bubble', 'bubble', 'bubble', 'bubble', 'bubble']
numids [1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1]
objects: ['bubble', 'bubble', 'bubble', 'bubble', 'bubble', 'bubble', 'chamber', 'chamber', 'chamber', 'chamber']
numids [1, 1, 1, 1, 1, 1, 2, 2, 2, 2]
objects: ['bubble',

/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2087: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'




Epoch 1/10


ERROR:root:Error processing image {'id': 'set2_61m.png', 'source': 'object', 'path': '/content/drive/MyDrive/APHIS Farm Bill (2020Milestones)/Protocols/For John/images/New set for John/collection/four_chambers/train/set2_61m.png', 'width': 2988, 'height': 5312, 'polygons': [{'name': 'ellipse', 'cx': 1027, 'cy': 2872, 'rx': 128, 'ry': 73, 'theta': 0}, {'name': 'ellipse', 'cx': 917, 'cy': 2078, 'rx': 67, 'ry': 49, 'theta': 0}, {'name': 'ellipse', 'cx': 1283, 'cy': 2132, 'rx': 49, 'ry': 67, 'theta': 0}, {'name': 'polygon', 'all_points_x': [1004, 1159, 1205, 1269, 1320, 1311, 1274, 1178, 1059, 953, 903, 862, 866, 880, 907], 'all_points_y': [2058, 2067, 2085, 2145, 2232, 2392, 2488, 2521, 2530, 2502, 2452, 2351, 2264, 2172, 2131]}, {'name': 'polygon', 'all_points_x': [1741, 1842, 1952, 2044, 2108, 2131, 2122, 2090, 1989, 1874, 1778, 1723, 1686, 1682, 1686, 1705], 'all_points_y': [2062, 2030, 2021, 2039, 2104, 2204, 2351, 2429, 2475, 2484, 2470, 2438, 2360, 2282, 2191, 2117]}, {'name': 'poly

KeyError: ignored